In [2]:
import csv
from ural import LRUTrie

 ### ETAPE 1 : PREPARATION DE LA BASE MEDIA_CLOUD

In [4]:

###Préparation de deux bases avant la concaténation###

media_loc=[]
media_nat=[]


with open('./local.csv') as loc:
    reader = csv.DictReader(loc, delimiter=',')

    for line in reader:
        media_loc.append(line)
        
    
with open('./national.csv') as nat:
    reader = csv.DictReader(nat, delimiter=',')
    
    for line in reader:
        media_nat.append(line)
     

In [5]:
###identification des doublons entre le national et le local dans mediacloud###

id_loc=[]
set_id_nat=set()

for media in media_loc:
    id_loc.append(media['url'])

for media in media_nat:
    set_id_nat.add(media['url'])
    
set_id_loc=set(media['url'] for media in media_loc)

In [6]:
###Suppression des doublons puis concaténation national et local dans media_cloud###

doublon = set_id_loc & set_id_nat

media_url={}

for media in media_loc:
    media_url[media['url']]=media

for url in doublon:
    media_loc.remove(media_url[url])


In [7]:
###vérification###

id_loc=[]
for media in media_loc:
    id_loc.append(media['url'])
    
set_id_loc=set(id_loc)
doublon = set_id_loc & set_id_nat
print(doublon)

set()


In [8]:
media_cloud=media_loc + media_nat

 ### ETAPE 2 

In [17]:
### identification de l'intersection mediacloud et hyphe###
### identification de ceux de mediacloud qui ne sont pas dans hyphe###

hyphe=[]

with open('./hyphe.csv') as hph:
    reader = csv.DictReader(hph, delimiter=',')
    for line in reader:
        if line['batch (TAGS)']!='EU':
            hyphe.append(line)

trie = LRUTrie(strip_trailing_slash=True)

match=[]

for media in hyphe:
    d = {'ID_hyphe' : media['ID'], 'Nom' : media['NAME'], 'startpage_hyphe' : media['HOME PAGE']}
    
    for line in media['PREFIXES AS URL'].split(" "):
        trie.set(line, d)
        
    match.append(d)
        
for media in media_cloud:
    d = {'id_mediacloud' : media['media_id'],'startpage_mediacloud' : media['url'], 'Nom_mediacloud': media['name']}
    m = trie.match(media['url'])
    
    if m is not None:
        m.update(d)
    else:  
        match.append(d)

In [20]:
#[media for media in match if 'id_mediacloud' not in media]

In [18]:
###identification de ceux de hyphe qui ne sont pas dans mediacloud###

In [21]:
with open('final.csv', 'w') as of:
    writer=csv.DictWriter(of, fieldnames=list(match[0].keys()))
    writer.writeheader()
    for media in match:
        writer.writerow(media)